dataset preparation for component2/3

In [ ]:
import pandas as pd
import os

def filter_data_by_game_ids():
    print("Reading game data...")
    ga_data = pd.read_csv('ga.csv')
    valid_game_ids = set(ga_data['app_id'])
    print(f"Total {len(valid_game_ids)} valid game IDs")

    print("\nProcessing review data (re.csv)...")

    output_re_file = 're_filtered.csv'
    output_us_file = 'us_filtered.csv'

    chunk_size = 10000
    first_chunk = True

    valid_user_ids_from_re = set()

    for chunk in pd.read_csv('re.csv', chunksize=chunk_size):
        filtered_chunk = chunk[chunk['app_id'].isin(valid_game_ids)]

        valid_user_ids_from_re.update(filtered_chunk['user_id'].unique())

        if first_chunk:
            filtered_chunk.to_csv(output_re_file, index=False, mode='w')
            first_chunk = False
        else:
            filtered_chunk.to_csv(output_re_file, index=False, mode='a', header=False)

        print(f"Processed {len(chunk)} records, kept {len(filtered_chunk)}")

    print(f"Review data saved to {output_re_file}")
    print(f"Collected {len(valid_user_ids_from_re)} valid user IDs from review data")

    print("\nProcessing user data (us.csv)...")

    first_chunk = True
    total_processed = 0
    total_kept = 0

    for chunk in pd.read_csv('us.csv', chunksize=chunk_size):
        total_processed += len(chunk)

        filtered_chunk = chunk[chunk['user_id'].isin(valid_user_ids_from_re)]
        total_kept += len(filtered_chunk)

        if first_chunk:
            filtered_chunk.to_csv(output_us_file, index=False, mode='w')
            first_chunk = False
        else:
            filtered_chunk.to_csv(output_us_file, index=False, mode='a', header=False)

        print(f"Processed {len(chunk)} user records, kept {len(filtered_chunk)}")

    print(f"\nUser data saved to {output_us_file}")
    print(f"Total processed {total_processed} user records, kept {total_kept} ({total_kept/total_processed*100:.1f}%)")

    print("\n=== Data filtering completed ===")
    print(f"Valid games: {len(valid_game_ids)}")
    print(f"Valid users: {len(valid_user_ids_from_re)}")

    for file_path in [output_re_file, output_us_file]:
        if os.path.exists(file_path):
            file_size = os.path.getsize(file_path) / (1024*1024)
            print(f"{file_path}: {file_size:.2f} MB")

def filter_data_by_game_ids_memory_optimized():
    import csv

    print("Reading game data...")
    ga_data = pd.read_csv('ga.csv')
    valid_game_ids = set(ga_data['app_id'])
    print(f"Total {len(valid_game_ids)} valid game IDs")

    print("\nProcessing review data (re.csv)...")
    re_output_file = 're_filtered.csv'

    valid_user_ids = set()
    re_count = 0
    kept_re_count = 0

    with open('re.csv', 'r', encoding='utf-8') as infile, \
         open(re_output_file, 'w', encoding='utf-8', newline='') as outfile:

        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames

        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        for row in reader:
            re_count += 1
            app_id = int(row['app_id']) if row['app_id'].isdigit() else row['app_id']

            if app_id in valid_game_ids:
                writer.writerow(row)
                kept_re_count += 1
                valid_user_ids.add(row['user_id'])

            if re_count % 10000 == 0:
                print(f"Processed {re_count} reviews, kept {kept_re_count}")

    print(f"Review data saved to {re_output_file}")
    print(f"Collected {len(valid_user_ids)} valid user IDs from review data")

    print("\nProcessing user data (us.csv)...")
    us_output_file = 'us_filtered.csv'

    us_count = 0
    kept_us_count = 0

    with open('us.csv', 'r', encoding='utf-8') as infile, \
         open(us_output_file, 'w', encoding='utf-8', newline='') as outfile:

        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames

        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        for row in reader:
            us_count += 1
            user_id = row['user_id']

            if user_id in valid_user_ids:
                writer.writerow(row)
                kept_us_count += 1

            if us_count % 10000 == 0:
                print(f"Processed {us_count} user records, kept {kept_us_count}")

    print(f"User data saved to {us_output_file}")

    print("\n=== Data filtering completed ===")
    print(f"Valid games: {len(valid_game_ids)}")
    print(f"Valid users: {len(valid_user_ids)}")
    print(f"Reviews: {re_count} -> {kept_re_count} ({kept_re_count/re_count*100:.1f}%)")
    print(f"Users: {us_count} -> {kept_us_count} ({kept_us_count/us_count*100:.1f}%)")

def split_by_game_id():
    print("Splitting review data by game ID...")

    ga_data = pd.read_csv('ga.csv')
    valid_game_ids = set(ga_data['app_id'])

    output_files = {}

    with open('re.csv', 'r', encoding='utf-8') as f:
        header = f.readline()

    chunk_size = 50000
    for chunk in pd.read_csv('re.csv', chunksize=chunk_size):
        filtered_chunk = chunk[chunk['app_id'].isin(valid_game_ids)]

        for game_id, group in filtered_chunk.groupby('app_id'):
            if game_id not in output_files:
                filename = f're_game_{game_id}.csv'
                output_files[game_id] = open(filename, 'w', encoding='utf-8')
                output_files[game_id].write(header)

            group.to_csv(output_files[game_id], index=False, header=False, mode='a')

    for f in output_files.values():
        f.close()

    print(f"Created {len(output_files)} files split by game ID")

if __name__ == "__main__":
    print("Please choose processing method:")
    print("1. Standard chunked processing (for large files)")
    print("2. Memory-optimized line-by-line processing (for very large files)")
    print("3. Split and save by game ID")

    choice = input("Enter option (1-3): ").strip()

    if choice == "1":
        filter_data_by_game_ids()
    elif choice == "2":
        filter_data_by_game_ids_memory_optimized()
    elif choice == "3":
        split_by_game_id()
    else:
        print("Using default standard chunked processing...")
        filter_data_by_game_ids()

useful game data filtering

In [ ]:
import pandas as pd
from pathlib import Path
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

def analyze_game_reviews(file_path, min_reviews=15):
    print(f"Starting to process file: {file_path}")
    print(f"File size: {Path(file_path).stat().st_size / (1024*1024):.2f} MB")

    def process_in_chunks():
        print("Processing large file using chunked reading...")

        stats = {}
        chunksize = 100000
        total_chunks = 0

        try:
            print("Calculating total number of rows in the file...")
            with open(file_path, 'r', encoding='utf-8') as f:
                total_rows = sum(1 for _ in f) - 1
            print(f"Total rows: {total_rows:,}")

            chunk_iter = pd.read_csv(file_path,
                                     chunksize=chunksize,
                                     encoding='utf-8',
                                     on_bad_lines='warn',
                                     low_memory=False)

            with tqdm(total=total_rows, desc="Processing Progress") as pbar:
                for chunk in chunk_iter:
                    total_chunks += 1

                    chunk.columns = chunk.columns.str.strip()

                    required_cols = ['app_id', 'recommendation']
                    missing_cols = [col for col in required_cols if col not in chunk.columns]
                    if missing_cols:
                        print(f"Warning: Missing required columns: {missing_cols}")
                        print(f"Available columns: {list(chunk.columns)}")
                        continue

                    for _, row in chunk.iterrows():
                        app_id = str(row['app_id']).strip()

                        if app_id not in stats:
                            stats[app_id] = {
                                'total_reviews': 0,
                                'recommended': 0,
                                'not_recommended': 0
                            }

                        stats[app_id]['total_reviews'] += 1

                        rec = str(row['recommendation']).strip().lower()
                        if rec == 'recommended':
                            stats[app_id]['recommended'] += 1
                        elif rec == 'not recommended':
                            stats[app_id]['not_recommended'] += 1

                    pbar.update(len(chunk))

            print(f"Processing complete, read {total_chunks} chunks")

        except FileNotFoundError:
            print(f"Error: File '{file_path}' does not exist")
            return None
        except Exception as e:
            print(f"Error reading file: {e}")
            return None

        return stats

    def process_directly():
        print("Processing file using direct reading...")
        try:
            df = pd.read_csv(file_path, encoding='utf-8', on_bad_lines='warn', low_memory=False)

            df.columns = df.columns.str.strip()

            required_cols = ['app_id', 'recommendation']
            for col in required_cols:
                if col not in df.columns:
                    print(f"Error: Missing required column '{col}'")
                    print(f"Available columns: {list(df.columns)}")
                    return None

            df['app_id'] = df['app_id'].astype(str).str.strip()
            df['recommendation'] = df['recommendation'].astype(str).str.strip().str.lower()

            stats = {}
            for app_id, group in df.groupby('app_id'):
                total = len(group)
                recommended = (group['recommendation'] == 'recommended').sum()
                not_recommended = (group['recommendation'] == 'not recommended').sum()

                stats[app_id] = {
                    'total_reviews': total,
                    'recommended': recommended,
                    'not_recommended': not_recommended
                }

            return stats

        except Exception as e:
            print(f"Error reading file: {e}")
            return None

    file_size_mb = Path(file_path).stat().st_size / (1024*1024)

    if file_size_mb > 50:
        stats = process_in_chunks()
    else:
        stats = process_directly()

    if stats is None:
        print("Statistics failed")
        return None

    results = []
    for app_id, data in stats.items():
        total = data['total_reviews']
        recommended = data['recommended']
        not_recommended = data['not_recommended']
        neutral = total - recommended - not_recommended

        results.append({
            'app_id': app_id,
            'total_reviews': total,
            'recommended': recommended,
            'not_recommended': not_recommended,
            'neutral': neutral,
            'recommendation_rate': recommended / total if total > 0 else 0,
            'not_recommendation_rate': not_recommended / total if total > 0 else 0,
            'neutral_rate': neutral / total if total > 0 else 0
        })

    df_results = pd.DataFrame(results)
    df_filtered = df_results[df_results['total_reviews'] >= min_reviews].copy()
    filtered_out = df_results[df_results['total_reviews'] < min_reviews]
    df_filtered = df_filtered.sort_values('total_reviews', ascending=False)

    return df_filtered, filtered_out

def save_results(df_filtered, filtered_out, output_dir='results'):
    Path(output_dir).mkdir(exist_ok=True)
    timestamp = time.strftime("%Y%m%d_%H%M%S")

    main_file = f"{output_dir}/game_review_stats_{timestamp}.csv"
    df_filtered.to_csv(main_file, index=False, encoding='utf-8-sig')

    if not filtered_out.empty:
        filtered_file = f"{output_dir}/filtered_games_{timestamp}.csv"
        filtered_out.to_csv(filtered_file, index=False, encoding='utf-8-sig')
        print(f"Filtered-out games saved to: {filtered_file}")

    total_reviews = df_filtered['total_reviews'].sum()
    total_recommended = df_filtered['recommended'].sum()
    total_not_recommended = df_filtered['not_recommended'].sum()
    total_neutral = df_filtered['neutral'].sum()

    summary = {
        'Analysis Time': time.strftime("%Y-%m-%d %H:%M:%S"),
        'Total Games Analyzed': len(df_filtered),
        'Games Filtered Out': len(filtered_out),
        'Total Reviews': total_reviews,
        'Total Recommended': total_recommended,
        'Total Not Recommended': total_not_recommended,
        'Total N/A Reviews': total_neutral,
        'Overall Recommendation Rate': f"{total_recommended/total_reviews*100:.1f}%",
        'Overall Not Recommendation Rate': f"{total_not_recommended/total_reviews*100:.1f}%",
        'Overall N/A Rate': f"{total_neutral/total_reviews*100:.1f}%",
        'Average Recommendation Rate': f"{df_filtered['recommendation_rate'].mean()*100:.1f}%",
        'Average Not Recommendation Rate': f"{df_filtered['not_recommendation_rate'].mean()*100:.1f}%",
        'Game with Highest Recommendation Rate': df_filtered.loc[df_filtered['recommendation_rate'].idxmax(), 'app_id'],
        'Highest Recommendation Rate': f"{df_filtered['recommendation_rate'].max()*100:.1f}%",
        'Game with Highest Not Recommendation Rate': df_filtered.loc[df_filtered['not_recommendation_rate'].idxmax(), 'app_id'],
        'Highest Not Recommendation Rate': f"{df_filtered['not_recommendation_rate'].max()*100:.1f}%",
        'Most Reviewed Game': df_filtered.loc[df_filtered['total_reviews'].idxmax(), 'app_id'],
        'Most Reviews': df_filtered['total_reviews'].max()
    }

    summary_file = f"{output_dir}/summary_{timestamp}.txt"
    with open(summary_file, 'w', encoding='utf-8') as f:
        f.write("Game Review Statistics Analysis Summary\n")
        f.write("=" * 60 + "\n")
        for key, value in summary.items():
            f.write(f"{key:25}: {value}\n")

    report_file = f"{output_dir}/detailed_report_{timestamp}.txt"
    with open(report_file, 'w', encoding='utf-8') as f:
        f.write("Detailed Game Review Statistics Analysis Report\n")
        f.write("=" * 100 + "\n\n")

        f.write("【Overall Statistics】\n")
        f.write("-" * 60 + "\n")
        f.write(f"Analyzed Games: {len(df_filtered):,}\n")
        f.write(f"Total Reviews: {total_reviews:,}\n")
        f.write(f"Recommended: {total_recommended:,} ({total_recommended/total_reviews*100:.1f}%)\n")
        f.write(f"Not Recommended: {total_not_recommended:,} ({total_not_recommended/total_reviews*100:.1f}%)\n")
        f.write(f"N/A: {total_neutral:,} ({total_neutral/total_reviews*100:.1f}%)\n\n")

        f.write("【Top 10 Games by Recommendation Rate】\n")
        f.write("-" * 100 + "\n")
        f.write(f"{'Rank':<4} {'Game ID':<15} {'Reviews':<10} {'Rec.':<10} {'Not Rec.':<12} {'Rec. Rate':<10} {'Not Rec. Rate':<10}\n")
        f.write("-" * 100 + "\n")

        top_recommended = df_filtered.nlargest(10, 'recommendation_rate')
        for i, (_, row) in enumerate(top_recommended.iterrows(), 1):
            f.write(f"{i:<4} {row['app_id']:<15} {row['total_reviews']:<10} {row['recommended']:<10} "
                    f"{row['not_recommended']:<12} {row['recommendation_rate']*100:>9.1f}% {row['not_recommendation_rate']*100:>9.1f}%\n")

        f.write("\n【Top 10 Games by Not Recommendation Rate】\n")
        f.write("-" * 100 + "\n")
        f.write(f"{'Rank':<4} {'Game ID':<15} {'Reviews':<10} {'Rec.':<10} {'Not Rec.':<12} {'Rec. Rate':<10} {'Not Rec. Rate':<10}\n")
        f.write("-" * 100 + "\n")

        top_not_recommended = df_filtered.nlargest(10, 'not_recommendation_rate')
        for i, (_, row) in enumerate(top_not_recommended.iterrows(), 1):
            f.write(f"{i:<4} {row['app_id']:<15} {row['total_reviews']:<10} {row['recommended']:<10} "
                    f"{row['not_recommended']:<12} {row['recommendation_rate']*100:>9.1f}% {row['not_recommendation_rate']*100:>9.1f}%\n")

        f.write("\n【Top 10 Most Reviewed Games】\n")
        f.write("-" * 100 + "\n")
        f.write(f"{'Rank':<4} {'Game ID':<15} {'Reviews':<10} {'Rec.':<10} {'Not Rec.':<12} {'Rec. Rate':<10} {'Not Rec. Rate':<10}\n")
        f.write("-" * 100 + "\n")

        top_reviewed = df_filtered.nlargest(10, 'total_reviews')
        for i, (_, row) in enumerate(top_reviewed.iterrows(), 1):
            f.write(f"{i:<4} {row['app_id']:<15} {row['total_reviews']:<10} {row['recommended']:<10} "
                    f"{row['not_recommended']:<12} {row['recommendation_rate']*100:>9.1f}% {row['not_recommendation_rate']*100:>9.1f}%\n")

    print(f"Main results saved to: {main_file}")
    print(f"Summary saved to: {summary_file}")
    print(f"Detailed report saved to: {report_file}")

    return main_file

def print_summary(df_filtered):
    if df_filtered is None or df_filtered.empty:
        print("No results to display")
        return

    total_reviews = df_filtered['total_reviews'].sum()
    total_recommended = df_filtered['recommended'].sum()
    total_not_recommended = df_filtered['not_recommended'].sum()
    total_neutral = df_filtered['neutral'].sum()

    print("\n" + "="*80)
    print("Game Review Statistics Results")
    print("="*80)
    print(f"Games meeting criteria: {len(df_filtered):,}")
    print(f"Total reviews: {total_reviews:,}")
    print(f"Recommended: {total_recommended:,} ({total_recommended/total_reviews*100:.1f}%)")
    print(f"Not Recommended: {total_not_recommended:,} ({total_not_recommended/total_reviews*100:.1f}%)")
    print(f"N/A: {total_neutral:,} ({total_neutral/total_reviews*100:.1f}%)")
    print("="*80)

    print("\nStatistics for the top 20 games:")
    print("-"*100)
    print(f"{'Game ID':<12} {'Reviews':<10} {'Rec.':<8} {'Not Rec.':<8} {'N/A':<8} {'Rec. Rate':<10} {'Not Rec. Rate':<10}")
    print("-"*100)

    for _, row in df_filtered.head(20).iterrows():
        rec_rate = row['recommendation_rate'] * 100
        not_rec_rate = row['not_recommendation_rate'] * 100
        print(f"{row['app_id']:<12} {row['total_reviews']:<10,} {row['recommended']:<8} "
              f"{row['not_recommended']:<8} {row['neutral']:<8} {rec_rate:>8.1f}% {not_rec_rate:>9.1f}%")

    if len(df_filtered) > 20:
        print(f"... {len(df_filtered) - 20} more games not shown")

if __name__ == "__main__":
    input_file = "game_review.csv"

    if not Path(input_file).exists():
        print(f"Error: File '{input_file}' does not exist")
        print("Please ensure the file path is correct")
        exit(1)

    MIN_REVIEWS = 15

    try:
        print("Starting analysis of game review data...")
        start_time = time.time()

        df_filtered, filtered_out = analyze_game_reviews(input_file, min_reviews=MIN_REVIEWS)

        if df_filtered is not None and not df_filtered.empty:
            print_summary(df_filtered)
            output_file = save_results(df_filtered, filtered_out)

            end_time = time.time()
            print(f"\nAnalysis complete! Time elapsed: {end_time - start_time:.2f} seconds")
            print(f"Results saved to '{output_file}'")

            print("\nKey statistics:")
            print(f"- Most reviewed game: {df_filtered.iloc[0]['app_id']} ({df_filtered.iloc[0]['total_reviews']:,} reviews)")
            print(f"- Game with highest recommendation rate: {df_filtered.loc[df_filtered['recommendation_rate'].idxmax()]['app_id']} ({df_filtered['recommendation_rate'].max()*100:.1f}%)")
            print(f"- Game with highest not recommendation rate: {df_filtered.loc[df_filtered['not_recommendation_rate'].idxmax()]['app_id']} ({df_filtered['not_recommendation_rate'].max()*100:.1f}%)")

            print("\nReview type distribution:")
            print(f" Recommended: {total_recommended:,} ({total_recommended/total_reviews*100:.1f}%)")
            print(f" Not Recommended: {total_not_recommended:,} ({total_not_recommended/total_reviews*100:.1f}%)")
            print(f" N/A: {total_neutral:,} ({total_neutral/total_reviews*100:.1f}%)")
        else:
            print("No qualifying game data found")

    except KeyboardInterrupt:
        print("\nProcess interrupted by user")
    except Exception as e:
        print(f"Error during execution: {e}")
        import traceback
        traceback.print_exc()